In [1]:
DIR = "norms"

import os
import bilby

import numpy as np

from glob import glob
from scipy import stats
from likelihood import Likelihood

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Define the parameters prior distributions
priors = dict(
    param_0=bilby.core.prior.Gaussian(1.63, 0.12, "param_0"),

    param_1=bilby.core.prior.Uniform(2.36, 2.38, "param_1"), 
    param_2=bilby.core.prior.Uniform(3e4, 4e4, "param_2"), 
    param_3=bilby.core.prior.Uniform(-1, 0, "param_3"),
    param_4=bilby.core.prior.Uniform(-1e4, 2e4, "param_4"),

    param_5=bilby.core.prior.Uniform(3.50, 3.51, "param_5"), 
    param_6=bilby.core.prior.Uniform(5e4, 6e4, "param_6"), 
    param_7=bilby.core.prior.Uniform(-1, 0, "param_7"), 
    param_8=bilby.core.prior.Uniform(-0.1, 0.1, "param_8"),
    param_9=bilby.core.prior.Uniform(-4e3, 0, "param_9"), 
    param_10=bilby.core.prior.Uniform(-1e4, 1e5, "param_10"),

    param_11=bilby.core.prior.Uniform(3.53, 3.56, "param_11"), 
    param_12=bilby.core.prior.Uniform(4e4, 6e4, "param_12"),
    
    param_13=bilby.core.prior.LogNormal(0, 0.05, "param_13"),
    param_14=bilby.core.prior.LogNormal(0, 0.05, "param_14"),
    param_15=bilby.core.prior.LogNormal(0, 0.05, "param_15"),
    param_16=bilby.core.prior.LogNormal(0, 0.069, "param_16"),
    param_17=bilby.core.prior.LogNormal(0, 0.079, "param_17"),
    param_18=bilby.core.prior.LogNormal(0, 0.10, "param_18"),
    param_19=bilby.core.prior.LogNormal(0, 0.10, "param_19"),
    param_20=bilby.core.prior.LogNormal(0, 0.10, "param_20"),
    param_21=bilby.core.prior.LogNormal(0, 0.10, "param_21"),
    param_22=bilby.core.prior.LogNormal(0, 0.10, "param_22"),
    param_23=bilby.core.prior.Uniform(0.5, 2, "param_23"),
    param_24=bilby.core.prior.Uniform(0.5, 2, "param_24"),
    param_25=bilby.core.prior.Uniform(0.5, 2, "param_25"),
    param_26=bilby.core.prior.Uniform(0.5, 2, "param_26"),
    param_27=bilby.core.prior.Uniform(0.5, 2, "param_27")
)

# Restrict processes to one thread only
os.environ['OMP_NUM_THREADS'] = '1'

# emcee variables
nsteps = 1000              # How many steps should each walker take?
nprocs = 6                 # How many Python processes do you want to allocate?
ntemps = 10                # How many temperatures do you want to use?
nwalkers = 2 * len(priors) # How many walkers do you want to use?


In [2]:
# Prepare the likelihood
likelihood = Likelihood( "12c_pg.azr", nprocs=nprocs)

# Run the sampler
result = bilby.run_sampler( likelihood=likelihood, priors=priors,
                            sampler="ptemcee", outdir="results/bilby/", label="ntemps_{}".format(ntemps), clean=True,
                            nwalkers=nwalkers, ntemps=ntemps, nsteps=nsteps, npool=nprocs,
                            niterations_per_check=1, autocorr_tol=1 )

09:01 bilby INFO    : Running for label 'ntemps_5', output will be saved to 'results/bilby/'
09:02 bilby INFO    : Analysis priors:
09:02 bilby INFO    : param_0=Gaussian(mu=1.63, sigma=0.12, name='param_0', latex_label='param_0', unit=None, boundary=None)
09:02 bilby INFO    : param_1=Uniform(minimum=2.36, maximum=2.38, name='param_1', latex_label='param_1', unit=None, boundary=None)
09:02 bilby INFO    : param_2=Uniform(minimum=30000.0, maximum=40000.0, name='param_2', latex_label='param_2', unit=None, boundary=None)
09:02 bilby INFO    : param_3=Uniform(minimum=-1, maximum=0, name='param_3', latex_label='param_3', unit=None, boundary=None)
09:02 bilby INFO    : param_4=Uniform(minimum=-10000.0, maximum=20000.0, name='param_4', latex_label='param_4', unit=None, boundary=None)
09:02 bilby INFO    : param_5=Uniform(minimum=3.5, maximum=3.51, name='param_5', latex_label='param_5', unit=None, boundary=None)
09:02 bilby INFO    : param_6=Uniform(minimum=50000.0, maximum=60000.0, name='par

1|0:00:04|nc:2.8e+02|a0:0.00-1.00|swp:0.48-0.86|n:nan<5000|t!nan(+nan,+nan)|q:inf|16.24ms/ev
2|0:00:08|nc:5.6e+02|a0:0.00-1.00|swp:0.46-0.73|n:nan<5000|t!nan(+nan,+nan)|q:inf|15.44ms/ev
3|0:00:12|nc:8.4e+02|a0:0.00-1.00|swp:0.49-0.65|n:nan<5000|t!nan(+nan,+nan)|q:inf|15.41ms/ev
4|0:00:17|nc:1.1e+03|a0:0.00-1.00|swp:0.41-0.56|n:nan<5000|t!nan(+nan,+nan)|q:inf|15.07ms/ev
5|0:00:21|nc:1.4e+03|a0:0.00-1.00|swp:0.36-0.49|n:nan<5000|t!nan(+nan,+nan)|q:inf|15.27ms/ev
6|0:00:25|nc:1.7e+03|a0:0.00-0.83|swp:0.31-0.45|n:nan<5000|t!nan(+nan,+nan)|q:inf|15.49ms/ev
7|0:00:30|nc:2.0e+03|a0:0.00-0.86|swp:0.28-0.42|n:nan<5000|t!nan(+nan,+nan)|q:inf|15.92ms/ev
8|0:00:35|nc:2.2e+03|a0:0.00-0.88|swp:0.25-0.40|n:nan<5000|t!nan(+nan,+nan)|q:inf|16.29ms/ev
9|0:00:39|nc:2.5e+03|a0:0.11-0.78|swp:0.24-0.37|n:nan<5000|t!nan(+nan,+nan)|q:inf|16.66ms/ev
10|0:00:44|nc:2.8e+03|a0:0.10-0.80|swp:0.22-0.35|n:nan<5000|t!nan(+nan,+nan)|q:inf|16.97ms/ev
11|0:00:49|nc:3.1e+03|a0:0.18-0.82|swp:0.21-0.33|n:nan<5000|t!nan(+na

09:10 bilby INFO    : Writing checkpoint and diagnostics
09:10 bilby INFO    : Finished writing checkpoint


91|0:08:01|nc:2.5e+04|a0:0.32-0.58|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|28.00ms/ev
92|0:08:07|nc:2.6e+04|a0:0.32-0.58|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|29.22ms/ev
93|0:08:13|nc:2.6e+04|a0:0.31-0.58|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|28.88ms/ev
94|0:08:19|nc:2.6e+04|a0:0.31-0.59|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|20.74ms/ev
95|0:08:24|nc:2.7e+04|a0:0.31-0.58|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|20.31ms/ev
96|0:08:30|nc:2.7e+04|a0:0.31-0.57|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|19.84ms/ev
97|0:08:35|nc:2.7e+04|a0:0.32-0.58|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|19.27ms/ev
98|0:08:40|nc:2.7e+04|a0:0.32-0.57|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.99ms/ev
99|0:08:45|nc:2.8e+04|a0:0.32-0.57|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.83ms/ev
100|0:08:50|nc:2.8e+04|a0:0.33-0.56|swp:0.03-0.05|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.47ms/ev
101|0:08:56|nc:2.8e+04|a0:0.33-0.56|swp:0.03-0.05|n:nan<500

09:20 bilby INFO    : Writing checkpoint and diagnostics
09:20 bilby INFO    : Finished writing checkpoint


208|0:18:02|nc:5.8e+04|a0:0.34-0.48|swp:0.01-0.03|n:nan<5000|t!nan(+nan,+nan)|q:inf|25.60ms/ev
209|0:18:08|nc:5.9e+04|a0:0.33-0.48|swp:0.01-0.03|n:nan<5000|t!nan(+nan,+nan)|q:inf|25.67ms/ev
210|0:18:13|nc:5.9e+04|a0:0.34-0.48|swp:0.01-0.02|n:nan<5000|t!nan(+nan,+nan)|q:inf|25.13ms/ev
211|0:18:18|nc:5.9e+04|a0:0.34-0.48|swp:0.01-0.02|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.24ms/ev
212|0:18:23|nc:5.9e+04|a0:0.34-0.48|swp:0.01-0.02|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.08ms/ev
213|0:18:28|nc:6.0e+04|a0:0.34-0.48|swp:0.01-0.02|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.04ms/ev
214|0:18:33|nc:6.0e+04|a0:0.34-0.48|swp:0.01-0.02|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.14ms/ev
215|0:18:38|nc:6.0e+04|a0:0.34-0.48|swp:0.01-0.02|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.29ms/ev
216|0:18:43|nc:6.0e+04|a0:0.34-0.48|swp:0.01-0.02|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.33ms/ev
217|0:18:49|nc:6.1e+04|a0:0.35-0.48|swp:0.01-0.02|n:nan<5000|t!nan(+nan,+nan)|q:inf|18.38ms/ev
218|0:18:54|nc:6.1e+04|a0:0.34-0.48|swp:0.01-0.02|